#Clone the repo

In [ ]:
!git clone https://github.com/Moda007/MethodicalSplit.git

In [ ]:
from MethodicalSplit import Functions as Fn
from MethodicalSplit.ExpModel import ExpModel

#Define experiment details

In [ ]:
DataSet, ModelName, group_idx, exp_idx, Avg, Sub, Rare, stratify, train = Fn.expDetails(Fn.all_exp)

#Colab

##Ignore Warnings

In [ ]:
import warnings
warnings.filterwarnings('ignore')

##Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Importing Packages and Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

#Dataset

##Importing Dataset

In [ ]:
from keras.datasets import fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [ ]:
print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

##Concatenate train and test dataset

In [ ]:
x_data = np.concatenate([x_train, x_test], axis=0)
y_data = np.concatenate([y_train, y_test], axis=0)
print("x_data shape:", x_data.shape)
print("y_data shape:", y_data.shape)

##Split test data (holdout)

In [ ]:
from sklearn.model_selection import train_test_split

x_data, x_test, y_data, y_test = train_test_split(x_data, y_data, train_size= 0.9, test_size= 0.1)

print("x_data shape:", x_data.shape)
print("x_test shape:", x_test.shape)
print("y_data shape:", y_data.shape)
print("y_test shape:", y_test.shape)

#Model (Loop)

##>>> Experiment

In [ ]:
looped = True
if looped:
  looped = 'looped_'
else:
  looped = ''

ds_path = '/content/drive/My Drive/Thesis Notebooks/' + DataSet + '/'
exp_path = ds_path + ModelName + '/Group_' + group_idx + '/Exp' + exp_idx + '/'

In [ ]:
def plotResults(DataSet, ModelName, group_idx, exp_idx, history, loop_no=''):
  # plot the loss and accuracy
  global exp_path

  loop_no = str(loop_no)
  plot_path = exp_path + 'plot'

  acc = history.history['acc']
  val_acc = history.history['val_acc']
  loss = history.history['loss']
  val_loss = history.history['val_loss']
  epochs = range(1, len(acc) + 1)

  plt.title('Training and validation accuracy')
  plt.plot(epochs, acc, 'red', label='Training acc')
  plt.plot(epochs, val_acc, 'blue', label='Validation acc')
  plt.legend()

  plt.savefig(f'{plot_path}/{loop_no}TrainValidAcc.jpg')

  plt.figure()
  plt.title('Training and validation loss')
  plt.plot(epochs, loss, 'red', label='Training loss')
  plt.plot(epochs, val_loss, 'blue', label='Validation loss')
  plt.legend()
  plt.savefig(f'{plot_path}/{loop_no}TrainValidAcc&Loss.jpg')

  plt.show()

#Looped Experiment

In [ ]:
#Define All Results array
All_results = []

for idx in range(10):
  
  global exp_path

  print('*****************************')
  print(f'Experiment {str(idx)} starts')
  print('*****************************')

  ##Data Preperation
  ###Splitting Data into 70% Train, 30% Validate

  x_train_X, y_train_X, x_valid_X, y_valid_X, x_test_X, y_test_X = Fn.prepareDataGT(x_data, x_test, y_data, y_test, stratify)

  print('x_train_X shape:', x_train_X.shape)
  print('x_valid_X shape:', x_valid_X.shape)
  print('x_test_X shape:', x_test_X.shape)

  ##Train
  thisModel = ExpModel(ModelName, DataSet, x_train_X, y_train_X, x_valid_X, y_valid_X, x_test_X, y_test_X)
  model, history, results = thisModel.trainModel()

  ##Store experiment results
  All_results.append(results)

  ##Model Saving
  filename = str(idx) + ModelName + '_' + DataSet + '_' + group_idx + '_'+ exp_idx
  model.save(exp_path + filename + '.h5')

  # plot the loss and accuracy
  plotResults(DataSet, ModelName, group_idx, exp_idx, history, idx)
  
  print('*****************************')
  print(f'Experiment {str(idx)} ends')
  print('*****************************')

  tf.keras.backend.clear_session()

##Check experiments results

In [ ]:
if len(All_results)==10:
  print('Results Are Complete')
else:
  print('Results record has issue!!!')

##Exporting Results to Excel

In [ ]:
!pip install xlsxwriter

###Create Excel with columns header

In [ ]:
import pandas as pd
from openpyxl import load_workbook

excel_name = ModelName + '.xlsx'
sheet_name = group_idx + '_' + exp_idx
filepath = ds_path + excel_name

column_titles = [['Idx', 'Train Acc', 'Valid Acc', 'Train Loss', 'Valid Loss',\
                 'Test Acc', 'Test Precision', 'Test Recall', 'Test F-score', 'Hamming Loss']]
df = pd.DataFrame(column_titles)

In [ ]:
# Create writer object with an engine xlsxwriter
writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
# Write data to an excel
df.to_excel(writer, sheet_name=sheet_name, index=False, header=None)
writer.save()

###Parse this experiment results

In [ ]:
for idx, result in enumerate(All_results):
  this_result = [idx] + result['train'] + result['test']
  this_result = np.array(this_result).reshape(1,-1)
  this_result = pd.DataFrame(this_result)

  writer = pd.ExcelWriter(filepath, engine='openpyxl')
  # try to open an existing workbook
  writer.book = load_workbook(filepath)
  # copy existing sheets
  writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)
  # read existing file
  reader = pd.read_excel(filepath)
  # write out the new sheet
  this_result.to_excel(writer, sheet_name=sheet_name, index=False, header=False, startrow=idx+1)

  writer.close()